In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split

/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 216, got 192
  return f(*args, **kwds)
/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 216, got 192
  return f(*args, **kwds)
/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [3]:
df = pd.read_csv('transformed_v2.csv', index_col=0)

In [4]:
df.head()

,duration,sex,age,country,education,undergrad_major,role,industry,experience,salary,...,Q50_Part_5,Q50_Part_6,Q50_Part_7,Q50_Part_8,"Q12_MULTIPLE_CHOICE_Advanced statistical software (SPSS, SAS, etc.)","Q12_MULTIPLE_CHOICE_Basic statistical software (Microsoft Excel, Google Sheets, etc.)","Q12_MULTIPLE_CHOICE_Business intelligence software (Salesforce, Tableau, Spotfire, etc.)","Q12_MULTIPLE_CHOICE_Cloud-based data software & APIs (AWS, GCP, Azure, etc.)","Q12_MULTIPLE_CHOICE_Local or hosted development environments (RStudio, JupyterLab, etc.)",Q12_MULTIPLE_CHOICE_Other
2,434,Male,30-34,Indonesia,Bachelor’s degree,Engineering (non-computer focused),Other,Manufacturing/Fabrication,5-10,"10-20,000",...,0,0,0,0,0,1,0,0,0,0
3,718,Female,30-34,United States of America,Master’s degree,"Computer science (software engineering, etc.)",Data Scientist,I am a student,0-1,"0-10,000",...,0,0,0,0,0,0,0,0,1,0
5,731,Male,22-24,India,Master’s degree,Mathematics or statistics,Data Analyst,I am a student,0-1,"0-10,000",...,1,0,0,0,1,0,0,0,0,0
7,959,Male,35-39,Chile,Doctoral degree,"Information technology, networking, or system ...",Other,Academics/Education,10-15,"10-20,000",...,0,1,0,0,0,1,0,0,0,0
8,1758,Male,18-21,India,Master’s degree,"Information technology, networking, or system ...",Other,Other,0-1,"0-10,000",...,1,0,0,0,0,0,0,0,1,0


In [14]:
tech_columns = [c for c in df.columns if (c.startswith("Q12") and c != 'Q12_MULTIPLE_CHOICE_Other') \
 or c.startswith("Q11") or c.startswith("Q15") \
 or c.startswith('Q16') or c.startswith('Q19') \
                or c.startswith('Q20') \
 or c.startswith('Q22') or c.startswith('Q32') \
 or c.startswith('Q37') or c.startswith('Q38')]

In [15]:
target = df['salary']
features = df[[c for c in df.columns if c != 'salary' and c in tech_columns]]

In [16]:
features_dummies = pd.get_dummies(features, dummy_na=True)
features_dummies.shape

(12956, 136)

In [17]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

In [18]:
lr = LogisticRegression(random_state=42, solver='lbfgs', multi_class='auto')

In [19]:
from sklearn.model_selection import cross_val_score
cross_val_score(lr, features_dummies, target, cv=5).mean()

/home/ptyshevskyi/envs/loc_env/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/home/ptyshevskyi/envs/loc_env/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/home/ptyshevskyi/envs/loc_env/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/home/ptyshevskyi/envs/loc_env/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/home/ptyshevskyi/envs/loc_env/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs faile

0.2879052810296695

In [24]:
rf = RandomForestClassifier(random_state=42, n_estimators=100)

In [26]:
cross_val_score(rf, features_dummies, target, cv=5).mean()

0.2737033616726661

In [28]:
rf.fit(features_dummies, target)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=42, verbose=0,
                       warm_start=False)

In [29]:
import numpy as np

In [32]:
imp_sorted = sorted(rf.feature_importances_, reverse=True)
feature_names = features_dummies.columns[np.argsort(rf.feature_importances_)[::-1]]

In [33]:
for feat, imp in zip(feature_names, imp_sorted):
    print(feat, imp)

Q38_Part_4 0.02549623976199661
Q12_MULTIPLE_CHOICE_Local or hosted development environments (RStudio, JupyterLab, etc.) 0.023732160210156995
Q38_Part_18 0.02315735286044925
Q37_nan 0.022521601104215326
Q11_Part_5 0.021117341962486828
Q20_Scikit-Learn 0.02077654443710496
Q15_Part_1 0.02071035974083297
Q11_Part_3 0.02067523121452553
Q38_Part_1 0.020287712987330408
Q17_Python 0.020209517011534832
Q11_Part_2 0.019812611000594765
Q22_Matplotlib 0.019672744894270254
Q11_Part_1 0.01899566517351932
Q38_Part_21 0.018729309773697328
Q32_Numerical Data 0.018526723679249648
Q37_Coursera 0.018424163106894702
Q38_Part_3 0.01786566158000955
Q38_Part_14 0.017689300262272032
Q11_Part_4 0.01740974208770689
Q15_Part_3 0.017289002678276613
Q12_MULTIPLE_CHOICE_Basic statistical software (Microsoft Excel, Google Sheets, etc.) 0.01693465539697963
Q17_nan 0.016836033453009772
Q32_nan 0.016245770991590205
Q15_Part_2 0.01622633667948087
Q20_nan 0.016200301230640316
Q22_nan 0.016111378067140945
Q38_Part_11 0.016